In [7]:
from typing import List
from collections import Counter

In [8]:
class Solutions:
    def frequencySort(self, nums: List[int]) -> List[int]:
        return sorted(nums, key = lambda x : (Counter(nums)[x], -x))
    def carPooling(self, trips: List[List[int]], capacity : int) -> bool:
        passengers = [0] * 1001
        for passenger, start, end in trips:
            passengers[start] += passenger
            passengers[end] -= passenger
        for passenger in passengers:
            capacity -= passenger
            if capacity < 0:
                return False
        return True

In [9]:
nums = [1, 1, 2, 2, 2, 3]
solve = Solutions()
ans = solve.frequencySort(nums)
ans

[3, 1, 1, 2, 2, 2]

In [9]:
%%writefile fibonacci_heap.py
import math

class FibonacciNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.parent = None
        self.child = None
        self.left = self
        self.right = self
        self.degree = 0
        self.mark = False

class FibonacciHeap:
    def __init__(self):
        self.min_node = None
        self.total_nodes = 0

    def insert(self, key, value):
        new_node = FibonacciNode(key, value)
        if not self.min_node:
            self.min_node = new_node
        else:
            self._add_to_root_list(new_node)
            if new_node.key < self.min_node.key:
                self.min_node = new_node
        self.total_nodes += 1
        return new_node

    def extract_min(self):
        if not self.min_node:
            return None
        min_node = self.min_node
        if min_node.child:
            child = min_node.child
            while True:
                next_child = child.right
                self._add_to_root_list(child)
                child.parent = None
                if child == min_node.child:
                    break
                child = next_child
        self._remove_from_root_list(min_node)
        if min_node == min_node.right:
            self.min_node = None
        else:
            self.min_node = min_node.right
            self._consolidate()
        self.total_nodes -= 1
        return min_node

    def decrease_key(self, node, new_key):
        if new_key > node.key:
            raise ValueError("New key is greater than current key")
        node.key = new_key
        parent = node.parent
        if parent and node.key < parent.key:
            self._cut(node, parent)
            self._cascading_cut(parent)
        if self.min_node is None or node.key < self.min_node.key:
            self.min_node = node

    def _add_to_root_list(self, node):
        if not self.min_node:
            self.min_node = node
        else:
            node.right = self.min_node.right
            node.left = self.min_node
            self.min_node.right.left = node
            self.min_node.right = node

    def _remove_from_root_list(self, node):
        node.left.right = node.right
        node.right.left = node.left

    def _consolidate(self):
        max_degree = int(math.log2(self.total_nodes)) + 1
        degree_table = [None] * max_degree
        current = self.min_node
        root_list = []
        while current not in root_list:
            root_list.append(current)
            current = current.right
        for root in root_list:
            degree = root.degree
            while degree_table[degree]:
                other = degree_table[degree]
                if root.key > other.key:
                    root, other = other, root
                self._link(other, root)
                degree_table[degree] = None
                degree += 1
            degree_table[degree] = root
        self.min_node = None
        for root in degree_table:
            if root:
                if not self.min_node:
                    self.min_node = root
                else:
                    self._add_to_root_list(root)
                    if root.key < self.min_node.key:
                        self.min_node = root

    def _link(self, child, parent):
        self._remove_from_root_list(child)
        child.parent = parent
        if not parent.child:
            parent.child = child
            child.left = child
            child.right = child
        else:
            child.left = parent.child
            child.right = parent.child.right
            parent.child.right.left = child
            parent.child.right = child
        parent.degree += 1
        child.mark = False

    def _cut(self, child, parent):
        parent.degree -= 1
        if parent.child == child:
            parent.child = child.right
        if parent.degree == 0:
            parent.child = None
        self._remove_from_root_list(child)
        self._add_to_root_list(child)
        child.parent = None
        child.mark = False

    def _cascading_cut(self, node):
        parent = node.parent
        if parent:
            if not node.mark:
                node.mark = True
            else:
                self._cut(node, parent)
                self._cascading_cut(parent)

Overwriting fibonacci_heap.py


In [10]:
import heapq
import time
from fibonacci_heap import FibonacciHeap  # Assuming the above code is in fibonacci_heap.py

def dijkstra_array(graph, start):
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    pq = [(0, start)]
    
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        
        if current_distance > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))
    
    return distances

def dijkstra_fibonacci(graph, start):
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    fib_heap = FibonacciHeap()
    nodes = {}
    
    for node in graph:
        if node == start:
            nodes[node] = fib_heap.insert(0, node)
        else:
            nodes[node] = fib_heap.insert(float('infinity'), node)
    
    while fib_heap.total_nodes > 0:
        min_node = fib_heap.extract_min()
        if min_node is None:
            break
        current_node = min_node.value
        current_distance = min_node.key
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                if nodes[neighbor] in fib_heap.min_node:
                    fib_heap.decrease_key(nodes[neighbor], distance)
                else:
                    nodes[neighbor] = fib_heap.insert(distance, neighbor)
    
    return distances

# Example graph
graph = {
    'A': {'B': 4, 'C': 2},
    'B': {'A': 4, 'C': 1, 'D': 5},
    'C': {'A': 2, 'B': 1, 'D': 8, 'E': 10},
    'D': {'B': 5, 'C': 8, 'E': 2, 'F': 6},
    'E': {'C': 10, 'D': 2, 'F': 3},
    'F': {'D': 6, 'E': 3}
}

# Compare runtimes
start_node = 'A'

start_time = time.time()
result_array = dijkstra_array(graph, start_node)
array_time = time.time() - start_time

start_time = time.time()
result_fibonacci = dijkstra_fibonacci(graph, start_node)
fibonacci_time = time.time() - start_time

print(f"Array-based implementation time: {array_time:.6f} seconds")
print(f"Fibonacci Heap implementation time: {fibonacci_time:.6f} seconds")

TypeError: argument of type 'FibonacciNode' is not iterable